In [1]:
from numba import jit
import numpy as np
import time
import os
import glob
from sys import getsizeof
import pandas as pd
from mem import *
import copy

In [2]:
from mymodules import *

In [ ]:
#作成データ
#stc->過去３０日間に終値が移動平均１００より大き日が２０日以上存在する
#RSI[14]
#a：安値＞ボリンジャー、bで指値価格変更->ボリンジャー[14]
#RSIとボリンジャーは複数用意するか
#ボリンジャーと安値の関係で指値価格変えることできるか
years=list(map(str,list(range(1996,2019))))#2019
#stdataのaxis=1のインデックスを作る
datatime={}
indexdict={'code':0,'exchange':1,'volume':2,'open':3,'high':4,'low':5,'close':6}
stindex=pd.Series(['code','exchange','volume','open','high','low','close'])
name=glob.glob('./stdata/*.npy')[0]
year=os.path.splitext(os.path.split(name)[-1])[0][-4::]
datatime[year],stdata=np.split(np.load(name),[1],axis=1)
pdtime=pd.Series(datatime[year][1001,0,:])
stdata=stdata.astype(np.float32)

for name in glob.glob('./stdata/*.npy')[1::]:
    starttime=time.time()
    year=os.path.splitext(os.path.split(name)[-1])[0][-4::]
    datatime[year],stdata1=np.split(np.load(name),[1],axis=1)
    pdtime=pdtime.append(pd.Series(datatime[year][1001,0,:]),ignore_index=True)
    stdata1=stdata1.astype(np.float32)
    stdata=np.concatenate([stdata,stdata1],axis=2)
    endtime=time.time()-starttime
    print(name,':',endtime)
stdata=np.transpose(stdata,[2,1,0])
stdata[np.isnan(stdata)]=0

./stdata/stdata1997.npy : 5.058912038803101
./stdata/stdata1998.npy : 4.662067890167236
./stdata/stdata1999.npy : 4.958720922470093
./stdata/stdata2000.npy : 5.05747389793396
./stdata/stdata2001.npy : 5.96628212928772
./stdata/stdata2002.npy : 6.601095914840698
./stdata/stdata2003.npy : 7.069071054458618
./stdata/stdata2004.npy : 6.939129114151001


In [4]:
stdata=np.load('./stdata1/stdata.npy')

In [5]:
#指標の期間
periods=[7,15,30,60,120,240]

In [25]:
%%time
#double ->f8 == float64
stRSI=RSI(stdata[:,6,:].astype('f8'),periods)
#5桁くらいの表示　０〜１００の指標はこれで大丈夫
stRSI=stRSI.astype(np.float16)
np.save('./stdata1/stRSI.npy',stRSI)

CPU times: user 28.1 s, sys: 15.5 s, total: 43.6 s
Wall time: 49.6 s


In [9]:
%%time
#BBand,ave
Bup,Bave,Blow=bbandy(stdata[:,6,:].astype('f8'),periods,3)

CPU times: user 1min 11s, sys: 16.3 s, total: 1min 27s
Wall time: 1min 50s


In [16]:
Bup=Bup.astype(np.float32)
Bave=Bave.astype(np.float32)
Blow=Blow.astype(np.float32)

In [18]:
np.save('./stdata1/Bup.npy',Bup)
np.save('./stdata1/Blow.npy',Blow)
np.save('./stdata1/Bave.npy',Bave)

In [ ]:
del stRSI

In [18]:
#5桁くらいの表示　０〜１００の指標はこれで大丈夫
stRSI=stRSI.astype(np.float16)

In [19]:
stRSI[1001,0,-50::]

array([ 54.125  ,  46.15625,  57.21875,  46.15625,  52.28125,  51.1875 ,
        43.875  ,  46.25   ,  53.84375,  57.625  ,  53.90625,  54.75   ,
        81.     ,  82.4375 ,  90.6875 ,  37.5    ,  41.3125 ,  45.84375,
        55.15625,  60.5625 ,  52.28125,  36.59375,  35.75   ,  54.40625,
        42.65625,  53.90625,  41.21875,  45.15625,  66.875  ,  37.5    ,
        42.46875,  41.96875,  55.46875,  38.71875,  71.125  ,  84.125  ,
        40.40625,  46.09375,  46.0625 ,  31.65625,  52.8125 ,  61.6875 ,
        57.84375,  43.9375 ,  72.6875 ,  65.1875 ,  36.03125,  59.90625,
         0.     ,   0.     ], dtype=float16)

In [20]:
getsizeof(stRSI)/1000000

651.120128

In [4]:
stdatas={}

In [5]:
for i in range(len(stindex)):
    stdatas[stindex[i]]=stdata[:,i,:]

In [53]:
np.save('stdatas.npy',stdatas)

In [3]:
stdatas=np.load('stdatas.npy')

In [6]:
type(stdatas)

dict

In [19]:
stdata=np.transpose(stdata,[2,1,0])

In [34]:
stdata.shape

(5426, 7, 10000)

In [ ]:
%%time
stockrange=np.array([0,100,200,500,700,1000,1500,2000,3000,5000,7000,10000,15000,20000,30000,50000,70000,100000,150000,200000,300000,500000,700000,1000000,1500000,2000000,3000000,5000000,7000000,10000000,15000000,20000000,30000000,50000000,9999000000000])
stoprange=np.array([0,30,50,80,100,150,300,400,500,700,1000,1500,3000,4000,5000,7000,10000,15000,30000,40000,50000,70000,100000,150000,300000,400000,500000,700000,1000000,1500000,3000000,4000000,5000000,7000000,10000000]) 
stockranges=np.tile(n101(stockrange),(stdaf.shape[0],1,codes))
stopranges=np.tile(n101(stoprange),(stdaf.shape[0],1,codes))

In [ ]:
%%time
stopH=stdaf[:,4,:]
stopL=stdaf[:,4,:]
for sr in range(len(stockrange)-1):
    stopH=np.where((stdaf[:,4,:]<stockranges[:,sr+1,:])&
                   (stdaf[:,4,:]>stockranges[:,sr,:])
                      ,stopH+stopranges[:,sr+1,:],stopH)
    stopL=np.where((stdaf[:,4,:]<stockranges[:,sr+1,:])&
                   (stdaf[:,4,:]>stockranges[:,sr,:])
                      ,stopL-stopranges[:,sr+1,:],stopL)
stopH=stopH[:,np.newaxis,:]
stopL=stopL[:,np.newaxis,:]
stopH1p=np.roll(stopH,1,axis=0)
stopL1p=np.roll(stopL,1,axis=0)

In [24]:
pdtime

0         1996/1/4
1         1996/1/5
2         1996/1/8
3         1996/1/9
4        1996/1/10
5        1996/1/11
6        1996/1/12
7        1996/1/16
8        1996/1/17
9        1996/1/18
10       1996/1/19
11       1996/1/22
12       1996/1/23
13       1996/1/24
14       1996/1/25
15       1996/1/26
16       1996/1/29
17       1996/1/30
18       1996/1/31
19        1996/2/1
20        1996/2/2
21        1996/2/5
22        1996/2/6
23        1996/2/7
24        1996/2/8
25        1996/2/9
26       1996/2/13
27       1996/2/14
28       1996/2/15
29       1996/2/16
           ...    
5396    2017/12/18
5397    2017/12/19
5398    2017/12/20
5399    2017/12/21
5400    2017/12/22
5401    2017/12/25
5402    2017/12/26
5403    2017/12/27
5404    2017/12/28
5405    2017/12/29
5406      2018/1/4
5407      2018/1/5
5408      2018/1/9
5409     2018/1/10
5410     2018/1/11
5411     2018/1/12
5412     2018/1/15
5413     2018/1/16
5414     2018/1/17
5415     2018/1/18
5416     2018/1/19
5417     201

In [25]:
a=np.random.rand(3,3,3)
b=np.random.rand(3,3,3)

In [27]:
%%timeit
a*b

The slowest run took 1752.12 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.07 µs per loop


In [11]:
getsizeof(stdata)/1000000

1519.280128

In [6]:
datatime[year][1001,0,].shape

(10000, 1, 247)

In [7]:
c=pd.Series(datatime[year][1001,0,:])

In [9]:
a=pd.Series([1])
a.append(pd.Series([0,1,2]),ignore_index=True)

0    1
1    0
2    1
3    2
dtype: int64

In [2]:
#作成データ
#stc->過去３０日間に終値が移動平均１００より大き日が２０日以上存在する
#RSI[14]
#a：安値＞ボリンジャー、bで指値価格変更->ボリンジャー[14]
#RSIとボリンジャーは複数用意するか
#ボリンジャーと安値の関係で指値価格変えることできるか
years=list(map(str,list(range(1996,2019))))#2019
stdata={}
datatime={}
#stdataのaxis=1のインデックスを作る
stindex=pd.Series(['code','exchange','volume','open','high','low','close'])

for name in glob.glob('./stdata/*.npy'):
    starttime=time.time()
    year=os.path.splitext(os.path.split(name)[-1])[0][-4::]
    datatime[year],stdata[year]=np.split(np.load(name),[1],axis=1)
    stdata[year]=stdata[year].astype(np.float32)
    endtime=time.time()-starttime
    print(name,':',endtime)


./stdata/stdata1996.npy : 2.7835679054260254
./stdata/stdata1997.npy : 3.101583957672119
./stdata/stdata1998.npy : 4.315847158432007
./stdata/stdata1999.npy : 4.179213047027588
./stdata/stdata2000.npy : 4.313995122909546
./stdata/stdata2001.npy : 4.194653034210205
./stdata/stdata2002.npy : 5.266273021697998
./stdata/stdata2003.npy : 9.730456829071045
./stdata/stdata2004.npy : 9.038580894470215
./stdata/stdata2005.npy : 10.578870058059692
./stdata/stdata2006.npy : 6.756126880645752
./stdata/stdata2007.npy : 4.884593963623047
./stdata/stdata2008.npy : 5.218302965164185
./stdata/stdata2009.npy : 5.538614988327026
./stdata/stdata2010.npy : 5.345909118652344
./stdata/stdata2011.npy : 8.711882829666138
./stdata/stdata2012.npy : 6.79465389251709
./stdata/stdata2013.npy : 9.661933898925781
./stdata/stdata2014.npy : 11.696789026260376
./stdata/stdata2015.npy : 11.959646940231323
./stdata/stdata2016.npy : 9.995668888092041
./stdata/stdata2017.npy : 9.009958982467651
./stdata/stdata2018.npy : 0.5

In [6]:
def show_objects_size(threshold, unit=2):
    """
    生きている全部の変数のサイズを表示する。
    globals()を対象にしている為この関数をモジュール化してimportしても機能しない。
    開発中の確認用に、同一モジュール内で実行すること。

    Parameters
    ----------
    threshold : int, float
        表示するサイズの下限しきい値。
        unitに応じた値にすること。
    unit : int
        表示するサイズの単位
        1: KB
        2: MB
        3: GB
    ex.
        100MB超の変数をGB単位で表示したい場合
        threshold=0.1, unit=3

    Returns
    -------
        なし
    """
    disp_unit = {0: 'bites', 1: 'KB', 2: 'MB', 3: 'GB'}
    # 処理中に変数が変動しないように固定
    globals_copy = globals().copy()
    for object_name in globals_copy.keys():
        size = compute_object_size(eval(object_name))
        if size > threshold:
            print('{:<15}{:.3f} {}'.format(object_name, size, disp_unit[unit]))

__builtins__   0.052 MB


In [16]:
#作成データ
#stc->過去３０日間に終値が移動平均１００より大き日が２０日以上存在する
#RSI[14]
#a：安値＞ボリンジャー、bで指値価格変更->ボリンジャー[14]
#RSIとボリンジャーは複数用意するか
#ボリンジャーと安値の関係で指値価格変えることできるか1
name=glob.glob('./stdata/*.npy')
data96=np.load(name[0])

In [17]:
#オブジェクト型からfloatに移行
#０列目を他のオブジェクトに入れる
#stdata(0>株式コード,1>取引所、２＞Volume,3>open,4>high,5>Low,6>close)
datatime,stdata=np.split(data96,[1],axis=1)
#datatimeを他の辞書として入れる→pd.Series
datatime=pd.Series(datatime[1001,0,:])

#16bitより32bitのが計算速度早い→最後にもう一度確認
stdata=stdata.astype(np.float32)


NameError: name 'st' is not defined